In [23]:
import math
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt

cover_dir = 'data/covers'
cover_paths = os.listdir(cover_dir)
len(cover_paths)


207572

In [15]:
cover_paths.sort()
print(cover_paths[:10])


['000195850X.jpe', '000215949X.jpe', '000217068X.jpe', '000255206X.jpe', '000255268X.jpe', '000255710X.jpe', '000414077X.jpe', '000472514X.jpe', '000638692X.jpe', '000638871X.jpe']


In [16]:
from PIL import Image
def load_images(data_root, image_paths, target_size):
    images = []
    for img_path in image_paths:
        path = os.path.join(data_root, img_path)
        img = Image.open(path)
        img = img.resize(target_size)
        images.append(np.array(img))
    return images


In [ ]:
n_imgs = 1000
target_size = (200, 200)

# we can't load many images this way
cover_images = load_images(cover_dir, cover_paths[:n_imgs], target_size)
len(cover_images)


In [25]:
# shuffle
np.random.shuffle(cover_images)

# set aside 10% for testing
n_test = math.floor(len(cover_images) * 0.1)
covers_train = cover_images[n_test:]
covers_test = cover_images[:n_test]
len(covers_train), len(covers_test)

(900, 100)